In [1]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.search import get_phrases_by_collection
from src.phrases.utils import generate_phrase_hash
from src.models import BCP47Language
from src.connections.gcloud_auth import get_firestore_client
from src.audio import get_voice_model, generate_translation_audio
from src.storage import read_from_gcs
from src.storage import PRIVATE_BUCKET, download_from_gcs
from src.llm_tools.image_generation import generate_phrase_image_prompt
from src.llm_tools.refine_story_translation import refine_story_translation
from src.images.generator import generate_image
from src.story import Story, StoryPhrase, get_story
from src.utils import load_json
from langcodes import Language
from google.cloud.firestore import FieldFilter
from src.wiktionary.lookup import fetch_wiktionary_entry
import json


In [ ]:
import json

with open("../outputs/wiktionary_dump/wiktionary_dump.json", "r", encoding="utf-8") as f:
    for step, line in enumerate(f):
        data = json.loads(line)
        print(data)
        break


In [2]:
import gzip

In [15]:
import gzip
import json
from tqdm import tqdm
from urllib.parse import quote
# Input and output file paths
input_file = "../outputs/wiktionary_dump/raw-wiktextract-data.jsonl.gz"  # Replace with your actual filename
output_file = 'first_5_items.json'

# Read first 5 items
items = []
with gzip.open(input_file, 'rt', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        if i > 100_000:
            break
        try:
            item = json.loads(line)
            word = item.get("word")
            
            if word and ("'" in word):
                lang_code = item.get("lang_code")
                lang = item.get("lang")
                url = f"https://en.wiktionary.org/wiki/{word}#{lang}"
                safe_url = url.replace(" ", "_")
                safe_url = quote(safe_url, safe=":#/")
                key = f"{lang_code}_{word}"

                entry = {
                    "key": key,
                    "word": word,
                    "lang": lang,
                    "lang_code": lang_code,
                    "url": safe_url}
                items.append(entry)
        except json.JSONDecodeError as e:
            print(f"Error parsing line {i}: {e}")

# Write to output file with nice formatting
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(items, f, indent=2, ensure_ascii=False)

print(f"Successfully extracted {len(items)} items to {output_file}")

100001it [00:08, 12115.86it/s]

Successfully extracted 28 items to first_5_items.json


In [ ]:
"Sam! Har du sett det här offentliga meddelandet om allemansrätten? Det öppnas en ny vandringsled imorgon."

In [10]:
fetch_wiktionary_entry(token="offentliga", language_code="sv")

2026-01-06 20:11:37 - audio-language-trainer - ERROR - <string>:19 - Wiktionary lookup failed for 'offentliga': 403
2026-01-06 20:11:37 - audio-language-trainer - ERROR - <string>:20 - Response: <!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; display: flex; flex-direction: row; flex-wrap: wrap; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { margin: 0 2em 2em 0; }
a img { border: 0; }
h1 { margin-top: 1em; font-size: 1.2em; }
.content-text { flex: 1; }
p { margin: 0.7em 0 1em 0; }
a { color: #0645ad; text-decoration: none; }
a:hover { text-decoration: underline; }
code { font-family: sans-serif; }
summary { font-weight: bold; cursor: pointer; }
details[open] { background

WiktionaryEntry(token='offentliga', language_code='sv', exists=False, url=None, section_anchor=None, lookup_variant=None)

In [ ]:
# client = get_firestore_client()

# docs = client.collection("phrases").stream()

# for doc in docs:
#     print(doc.id)
#     if doc.id.startswith("story_"):
#         print(f"deleting {doc.id}")
#         doc.reference.delete()
#     # if doc.get('key').startswith("story_"):
#     #     print(doc.to_dict())
#         # doc.reference.delete()

In [8]:
story_title = "A Test Story"
story_summary="A brief summary of the test story."
story_title_hash = generate_phrase_hash(story_title)
story_dialogue = load_json("../data/test_story.json")

story_title2 = "A Test Story 2"
story_summary2="A brief summary of the test story 2."
story_title_hash2 = generate_phrase_hash(story_title2)
story_dialogue2 = load_json("../data/test_story2.json")



In [9]:
ts1 = Story.create(story_title, story_summary, story_dialogue, collection="Test", deck="Pack01")
ts2 = Story.create(story_title2, story_summary2, story_dialogue2, collection="Test2", deck="Pack01")
ts1.upload(overwrite=True)
ts2.upload(overwrite=True)

2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py:410 - Uploading phrase story_a_test_story_232dea#part_1#0#hey_sam_did_you_book_the_tickets_for_the_hiking_tr_5a2e92 with all translations to Firestore and GCS
2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py:1047 - Uploading all multimedia for en-GB translation
2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py:410 - Uploading phrase story_a_test_story_232dea#part_1#1#yes_i_just_signed_the_forms_yesterday_its_going_to_9a3ca4 with all translations to Firestore and GCS
2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py:1047 - Uploading all multimedia for en-GB translation
2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py:410 - Uploading phrase story_a_test_story_232dea#part_2#0#sam_wake_up_we_need_to_leave_in_thirty_minutes_57df1c with all translations to Firestore and GCS
2026-01-04 15:15:53 - audio-language-trainer - INFO - phrase_model.py

In [10]:
ts1 = get_story(story_title_hash = story_title_hash)
ts2 = get_story(story_title_hash= story_title_hash2)

In [12]:
ts1.publish_story("fr-FR", overwrite=True)
ts1.publish_story("it-IT", overwrite=True)
ts2.publish_story("fr-FR", overwrite=True)
ts2.publish_story("it-IT", overwrite=True)

2026-01-04 15:19:18 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists


2026-01-04 15:19:18 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists
2026-01-04 15:19:19 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists
2026-01-04 15:19:19 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists
2026-01-04 15:19:20 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists
2026-01-04 15:19:21 - audio-language-trainer - INFO - phrase_model.py:342 - Translation for fr-FR already exists
2026-01-04 15:19:22 - audio-language-trainer - INFO - phrase_model.py:1176 - Audio already exists for fr-FR story normal, skipping generation
2026-01-04 15:19:22 - audio-language-trainer - INFO - phrase_model.py:1176 - Audio already exists for fr-FR story normal, skipping generation
2026-01-04 15:19:22 - audio-language-trainer - INFO - phrase_model.py:1176 - Audio already exists for fr-FR story normal, skipping generation
2026-01-0

'https://storage.googleapis.com/audio-language-trainer-stories/stories/en-GB/it-IT/a_test_story_2_a5e6f2/index.html'

In [ ]:
ts1 = Story.create(story_title, story_summary, story_dialogue, collection="Test", deck="Pack01")

In [ ]:
ts1.translate("fr-FR")
ts1.upload()

In [ ]:
ts1.publish_story("fr-FR")

In [ ]:
ts1.upload()

In [ ]:
ts2 = Story.create(story_title, story_summary, story_dialogue, collection="Test2", deck="Pack01")

In [ ]:
ts2.translate("it-IT")
ts2.upload()
ts2.publish_story("it-IT")


In [ ]:
ts2.publish_story("fr-FR", overwrite=True)
ts1.publish_story("fr-FR", overwrite=True)
ts1.publish_story("it-IT", overwrite=True)

In [ ]:

ts2.publish_story()
ts2.upload()

In [ ]:
ts.decks.update({"Test" : ['deck1']})

In [ ]:
ts.collections.append("Test")


In [ ]:
ts.collections

In [ ]:
ts.translate("fr-FR")
ts.translate("it-IT")

In [ ]:
ts.published

In [ ]:
ts.publish_story("fr-FR")
ts.publish_story("it-IT")

In [ ]:
ts.upload()

In [ ]:
ts.story_parts['part_2'][1].story_phrase.translations['fr-FR'].text

In [ ]:
ts.upload(overwrite=True)

In [ ]:
ts.generate_audio(language="fr-FR", overwrite=True)

In [ ]:
ts.publish_story(target_language="fr-FR", overwrite=True)

In [ ]:
ts.upload()

In [ ]:
import os
os.path.dirname('../outputs/gcs\\audio-language-trainer-stories\\stories/en-GB/fr-FR/a_test_story_232dea.html')

In [ ]:
ts.story_parts['part_1'][0].story_phrase.translations


In [ ]:
ts.translate(target_language="fr-FR")

In [ ]:
ts.upload()

In [ ]:
ts.get_story_translation("fr-FR")

In [ ]:
from jinja2 import Environment, FileSystemLoader

In [ ]:
loader = FileSystemLoader("../src/templates")
env = Environment(loader=loader, autoescape=False)
template = env.get_template("story_template.html")

In [ ]:
html_string = template.render(ts.model_dump())
with open("../src/templates/test_story/test_story.html", "w", encoding="utf-8") as f:
    f.write(html_string)

In [ ]:
ts.model_dump()

In [ ]:
for part, utterances in ts.story_parts.items():
    for utterance in utterances:
        print(utterance)

In [ ]:
l.language_name()

In [ ]:
l = BCP47Language.get("en-G5B")

In [ ]:
ts = get_story("A Test Story")

In [ ]:
ts.translate(BCP47Language("sv-SE"), refine=True, overwrite=True)

In [ ]:
aud = ts.generate_audio(BCP47Language("sv-SE"))

In [ ]:
ts.upload(language=BCP47Language("sv-SE"), overwrite=True)

In [ ]:
ts._get_dialogue_with_translations(target_language=BCP47Language("sv-SE"))

In [ ]:
ts.model_dump()

In [ ]:
ts._get_dialogue_with_translations(BCP47Language("fr-FR"))

In [ ]:
ts.translate(BCP47Language("fr-FR"))

In [ ]:
ts.generate_audio(BCP47Language("fr-FR"))

In [ ]:
ts.upload(language=BCP47Language("fr-FR"), overwrite=True)

In [ ]:
story_json = ts._get_dialogue_with_translations(BCP47Language("fr-FR"))

In [ ]:
aud = ts._load_audio("sv-SE")

In [ ]:
aud['part_3'][2]

In [ ]:
from src.utils import save_json, load_json
import json

new_story =load_json("../data/test_story_translated.json", )

In [ ]:
new_story = refine_story_translation(story_json, language=BCP47Language("fr-FR"))

In [ ]:
ts._replace_translations(new_story, BCP47Language("fr-FR"))

In [ ]:
ts._get_dialogue_with_translations(BCP47Language("fr-FR"))

In [ ]:
story_json

In [ ]:

ts = Story.create(title=story_title, summary=story_summary, story_dialogue=story_dialogue)

In [ ]:
new_story

In [ ]:
ts.upload_phrase_entries()

In [ ]:
ts.model_dump()

In [ ]:
story_name = "Test Story"

In [ ]:
new_story  = Story.create(
    title=story_name,
    story_parts=story_dialogue)

In [ ]:
from google.cloud.firestore_v1 import FieldFilter
p = get_phrases_by_collection(["Pack01"])


In [ ]:
len(p)

In [ ]:
phrase = "Do you collect rare coins?"

prompt = generate_phrase_image_prompt(phrase)


In [ ]:
style = "new_default"
# Generate the image using available providers
image = generate_image(
    prompt=prompt,
    style=style
)
image

In [ ]:
image

In [ ]:
WarmUp150_phrase = download_from_gcs(bucket_name=PRIVATE_BUCKET, file_path="collections/WarmUp150/phrases.json")

In [ ]:
phrases = get_phrases_by_collection("WarmUp150")

In [ ]:
phrases[0].translations['fr-FR'].get_wiktionary_links()

In [ ]:
phrases[0].translations['fr-FR'].tokens

In [ ]:
#french translation text

french_translations = download_from_gcs(bucket_name=PRIVATE_BUCKET, file_path="collections/WarmUp150/french/translations.json")

In [ ]:
from itertools import islice
from tqdm import tqdm
WARUMUP_PHRASES = []

for old_hash, phrase_dict in tqdm(french_translations.items()):
    english_text = phrase_dict["english"]
    french_text = phrase_dict["french"]

    p = Phrase.create(english_text)
    p.collections.append('WarmUp150')
    p.translate('fr-FR', translated_text=french_text)
    p.upload()
    WARUMUP_PHRASES.append(p)

In [ ]:
WARUMUP_PHRASES[5].model_dump()

In [ ]:
for p in WARUMUP_PHRASES:
    p.download()

In [ ]:
WARUMUP_PHRASES[5].translations['fr-FR'].image

In [ ]:
for phrase in WarmUp150_phrase[:10]:
    p1 = Phrase.create(WarmUp150_phrase[0])
    p1.collections.append('WarmUp150')
    p1.translate('fr-FR')a
    p1.generate_audio()

In [ ]:
p1.translate(BCP47Language('fr-FR'))

In [ ]:
p1.upload()

In [ ]:
p1.generate_audio("flashcard", BCP47Language('fr-FR'))

In [ ]:
p1.get_audio('fr-FR', "flashcard", "slow")

In [ ]:
p1.translations['fr-FR'].audio[0].model_dump()

In [ ]:
p1.generate_audio(context="flashcard", language=BCP47Language('fr-FR'))

In [ ]:
p1.translations['fr-FR'].audio[0].audio_segment

In [ ]:
p1.upload()

In [ ]:
p1 = get_phrase('lets_eat_lunch_tomorrow_b765fb')

In [ ]:
p1.model_dump()

In [ ]:
p1.generate_audio(context="flashcard", language="fr-FR")


In [ ]:
p1.tran

In [ ]:
phrase.translations[0].firestore_document_ref

In [ ]:
phrase.translations[1]._upload_to_gcs()

In [ ]:

# Add German translation
de = phrase.translate(BCP47Language.get("de-DE"), refine=True)
print(f"✓ German: {de.text}")


In [ ]:
# Generate audio for each translation using the new generate_audio() method
# For flashcard context: generates slow and normal speeds
# For story context: generates normal and fast speeds

# French audio
print("Generating French audio (flashcard)...")
fr.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ French audio generated: {len(fr.audio)} audio files")
for audio in fr.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

# German audio
print("\nGenerating German audio (flashcard)...")
de.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ German audio generated: {len(de.audio)} audio files")
for audio in de.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

In [ ]:
# Get the English translation and generate image
en_gb = phrase.translations[0]  # The en-GB translation created in create_phrase()
print(f"✓ English translation: {en_gb.text}")

# Generate image for the English translation (shared across all language variants)
image_prompt = "A cheerful greeting scene with two people waving hello, bright morning sunlight"
image = generate_image(image_prompt, style="ghibli", project_id="swedish-course")
en_gb.image = image
en_gb.upload_image("audio-language-trainer-private-content")
print(f"✓ Image generated and attached: {en_gb.image_file_path}")

In [ ]:
# Test phrase generation with small vocab_dict
from src.phrases.generation import generate_phrases_from_vocab_dict

# Small test vocab_dict with a few verbs and vocabs
test_vocab_dict = {
    "verbs": ["want", "go", "see"],
    "vocab": ["apple", "table", "red", "big", "old", "door", "window", "house"]
}

print("=" * 60)
print("TESTING PHRASE GENERATION")
print("=" * 60)
print(f"\nInput vocab_dict:")
print(f"  Verbs: {test_vocab_dict['verbs']}")
print(f"  Vocab: {test_vocab_dict['vocab']}")
print(f"\nGenerating phrases...")
print("-" * 60)

try:
    phrases, tracking = generate_phrases_from_vocab_dict(
        test_vocab_dict, 
        max_iterations=1  # Just one iteration for testing
    )
    
    print(f"\n✓ Generation complete!")
    print(f"\nResults:")
    print(f"  Total phrases generated: {tracking['total_phrases']}")
    print(f"  Verb phrases: {tracking['verb_phrases']}")
    print(f"  Vocab phrases: {tracking['vocab_phrases']}")
    print(f"  Verbs processed: {tracking['verbs_processed']}")
    print(f"  Vocab processed: {tracking['vocab_processed']}")
    print(f"  Additional words found: {len(tracking['words_used'])}")
    
    if tracking['errors']:
        print(f"\n⚠ Errors encountered:")
        for error in tracking['errors']:
            print(f"  - {error}")
    
    print(f"\nGenerated Phrases (first 20):")
    for i, phrase in enumerate(phrases[:20], 1):
        print(f"  {i}. {phrase}")
    
    if len(phrases) > 20:
        print(f"  ... and {len(phrases) - 20} more phrases")
    
    print(f"\nAdditional words tracked:")
    print(f"  {tracking['words_used']}")
    
except Exception as e:
    print(f"\n✗ Error during phrase generation:")
    print(f"  {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()